## Notebook created for question1

In [1]:
import pandas as pd
import numpy as np
import requests

print("Libraries downloaded")

Libraries downloaded


## About the Data, Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

### is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

### Scraping table from HTML using BeautifulSoup, write a Python program similar to scrape.py,from:

In [7]:

import requests
!pip install --upgrade bs4
from bs4 import BeautifulSoup
import ssl
import csv

print("BeautifulSoup & csv imported")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=f11079401bfb4947e1f4e86ebf66e7160413db0f3d7a410d907fe122cfbaf32c
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
BeautifulSoup & csv imported


In [8]:
!pip install lxml
!pip install html5lib
import lxml
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,"html.parser")
table = soup.find('table')

lst_table=[]
for line in table.find_all('tr'):
    if(line.contents[3].text != 'Not Assigned'):
        lst_table.append([line.contents[1].text, line.contents[3].text, line.contents[5].text[:-1]])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 117 kB 3.4 MB/s eta 0:00:01


In [9]:
df_postal = pd.DataFrame(lst_table[1:], columns=lst_table[0])
df_code = df_postal['Postcode'].unique()
df_postal.set_index('Postcode', drop=False, inplace=True)

separator=', '
for postcode in df_code:
    df_tmp = df_postal.loc[postcode]
    if(df_tmp.size > 3):
        neighbourhood = separator.join(df_tmp['Neighbourhood'])
        df_postal.loc[postcode, 'Neighbourhood'] = neighbourhood

df_postal.drop(df_postal[df_postal.Neighbourhood == 'Not assigned'].index, inplace=True)
df_postal.drop_duplicates('Postcode', inplace=True)
df_postal.reset_index(drop = True, inplace=True)
df_postal.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [10]:
print('The dataframe shape is: ', df_postal.shape)

The dataframe shape is:  (102, 3)


# Question 2

## Let's add latitude and longitude for our Postal codes using this csv file: http://cocl.us/Geospatial_data

In [11]:
!pip install wget

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9682 sha256=a2617b36a8f78a6296cc3205cb074747e2ab6d3768fae951d868d50c2650d9c4
  Stored in directory: /home/jovyan/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [12]:
#!wget -O geospatial_data.csv 'http://cocl.us/Geospatial_data'
import wget
url = 'http://cocl.us/Geospatial_data'
filename = wget.download(url)
filename

'Geospatial_Coordinates.csv'

In [13]:
geospatial_data = pd.read_csv(filename)
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
geospatial_data = geospatial_data.rename(columns={"Postal Code": "Postcode"})
geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_postal = df_postal.merge(geospatial_data, on='Postcode')
                                                        
df_postal.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_postal['Borough'].unique()),
        df_postal.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


# Question 3

In [17]:
!pip install geopy

from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 104 kB 3.6 MB/s eta 0:00:01
The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Lets visualize Toronto Neighbourhoods

In [18]:
!pip install folium


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 91 kB 3.6 MB/s eta 0:00:011


In [21]:
import folium
print(folium.__version__)

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_postal['Latitude'], df_postal['Longitude'], df_postal['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

0.10.1
